# CBOW(Continuous bag-of-words)
> 참고: 책 <<밑바닥부터 시작하는 딥러닝2>>, pp.132-141

In [6]:
CORPUS = 'You say goodbye and I say hello.'

말뭉치를 단어 ID로 변환한다.

In [7]:
import numpy as np

# 변환 함수
def preprocess(corpus):
    # 소문자 변환
    corpus = corpus.lower()
    
    # 마침표 앞에 공백 추가
    corpus = corpus.replace('.', ' .')
    
    # 공백 기준으로 단어 나누기
    words = corpus.split()
    
    word_to_id = {}
    id_to_word = {}
    now_id = 0
    for w in words:
        if w not in word_to_id:
            word_to_id[w] = now_id
            id_to_word[now_id] = w
            now_id += 1

    # 각 단어에 대한 id
    ids = np.array([word_to_id[w] for w in words])

    return ids, word_to_id, id_to_word

In [8]:
ids, word_to_id, id_to_word = preprocess(CORPUS)

In [14]:
id_to_word

{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}

맥락과 타겟을 만든다.

In [22]:
def create_target_contexts(ids, window_size=1):
    '''
    window_size: target의 좌우 몇 개의 단어를 고려할 것인가?
    '''

    # 타겟
    target = ids[window_size:-window_size]

    # 맥락
    contexts = []
    for i in range(window_size, len(ids) - window_size):
        contexts.append([ids[i - window_size], ids[i + window_size]])
    
    return np.array(target), np.array(contexts)


In [23]:
target, contexts = create_target_contexts(ids)

In [24]:
print(target)
print(contexts)

[1 2 3 4 1 5]
[[0 2]
 [1 3]
 [2 4]
 [3 1]
 [4 5]
 [1 6]]
